#### Test mynds packages in the sandbox

In [ ]:
import ast

from pathlib import Path
from typing import Any, Callable, Optional

import Metashape as ms
import polars as pl

import mynd.backend.metashape as backend

from mynd.io import read_data_frame
from mynd.tasks.ingestion import map_metadata_to_cameras
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


from mynd.collections import CameraGroup


GroupID = CameraGroup.Identifier


def insert_metadata(identifier: GroupID, metadata: pl.DataFrame) -> None:
    """TODO"""

    camera_metadata: dict[str, dict] = map_metadata_to_cameras(
        metadata,
        label_column="label",
        data_columns=["exposure"],
    )

    # NOTE: Write your code here
    match backend.camera_services.update_camera_metadata(identifier, metadata):
        case Ok(updated):
            logger.info("successfully update metadata")
        case Err(message):
            logger.error(message)


def retrieve_metadata(identifier: GroupID) -> dict:
    """Retrieves metadata from a group."""

    match backend.camera_services.get_camera_metadata(identifier):
        case Ok(metadata):
            logger.info(metadata)
        case Err(message):
            logger.error(message)


def main() -> None:
    """Main entrypoint."""

    PATHS: dict[str, Path] = {
        "source": Path(
            "/data/kingston_snv_01/acfr_metashape_projects_dev/r23685bc_lite_metadata.psz"
        ),
        "destination": Path(
            "/data/kingston_snv_01/acfr_metashape_projects_dev/r23685bc_lite_metadata.psz"
        ),
        "metadata": Path(
            "/data/kingston_snv_01/acfr_cameras_metadata/r23685bc_20100605_021022_camera_metadata.csv"
        ),
    }

    # Options
    save: bool = False
    identifier: GroupID = GroupID(key=0, label="r23685bc_20100605_021022")

    metadata: pl.DataFrame = read_data_frame(PATHS.get("metadata")).unwrap()

    if not backend.loaded_project():
        match backend.load_project(PATHS.get("source")):
            case Ok(path):
                logger.info(f"loaded project: {path}")
            case Err(message):
                logger.error(message)
                return

    insert_metadata(identifier, metadata)
    retrieve_metadata(identifier)

    if save:
        match backend.save_project(PATHS.get("destination")):
            case Ok(path):
                logger.info(f"successfully saved project to: {path}")
            case Err(message):
                logger.error(f"failed to save project: {message}")


# INVOKE MAIN
main()